In [ ]:
import sys
sys.path.append('..')

from spyral.core.constants import QBRHO_2_P
from spyral.core.run_stacks import form_run_string

from spyral_utils.nuclear import NuclearDataMap
from spyral_utils.nuclear.target import GasTarget, load_target
from spyral_utils.plot import Histogrammer

from pathlib import Path
from scipy.constants import physical_constants, torr
from scipy.integrate import quad
from scipy.stats import iqr
import polars as pl
import numpy as np
import matplotlib.pyplot as plt
import vector
import lmfit

%matplotlib widget

In [ ]:
# Load data config
workspace_path = Path("D:\\e20009_analysis")
target_material_path = Path("C:\\Users\\zachs\\Desktop\\e20009_analysis\\e20009_analysis\\e20009_parameters\\e20009_target.json")

solver_result_path = workspace_path / "InterpSolver"
beam_events_path = workspace_path / "beam_events"

# Run number range (inclusive)
run_min = 108
run_max = 366

# The nucleus we observe (the one fitted)
ejectile_z = 1
ejectile_a = 2

# The incoming nucleus (the beam)
projectile_z = 4
projectile_a = 10

# The target nucleus
target_z = 1
target_a = 2

residual_z = target_z + projectile_z - ejectile_z
residual_a = target_a + projectile_a - ejectile_a

if residual_z < 0:
    raise Exception(f"Illegal nuclei! Residual Z: {residual_z}")
if residual_a < 1:
    raise Exception(f"Illegal nuclei! Residual A: {residual_a}")

In [ ]:
# Setup nuclear data objects
nuclear_map = NuclearDataMap()

target_material = load_target(target_material_path, nuclear_map)
if not isinstance(target_material, GasTarget):
    print('Target error!')

ejectile = nuclear_map.get_data(ejectile_z, ejectile_a)
projectile = nuclear_map.get_data(projectile_z, projectile_a)
target = nuclear_map.get_data(target_z, target_a)
residual = nuclear_map.get_data(residual_z, residual_a)
print(f"Reaction: {target}({projectile}, {ejectile}){residual}")
print(f"Target material: {target_material.ugly_string}")

In [ ]:
# Define parameters used for analysis

proj_energy_start = 93.1        # Units of MeV. Initial beam energy

min_z = 0.004    # Units of meters. Minimum z value of vertex (inclusive)
max_z = 0.958    # Units of meters. Maximum z value of vertex (inclusive)

min_cm = 18.0    # Units of degrees. Minimum center-of-mass scattering angle for events in excitation spectrum (inclusive)
max_cm = 60.0   # Units of degrees. Maximum center-of-mass scattering angle for events in excitation spectrum (inclusive)

hist_en_min = -2.0        # Smallest allowed excitation energy in histogram (inclusive)
hist_en_max = 5.25    # Largest allowed excitation energy in histogram (inclusive)
hist_en_bins = 137        # Number of bins in excitation energy histogram

ang_dist_bins = 21     # Number of bins in angular distribution

In [ ]:
# Define parameters used for finding the cross section

downscale_factor = 1000

ic_min_val = 300.0
ic_max_val = 850.0

boltzmann = physical_constants["Boltzmann constant"][0]    # Units of Joules / Kelvin
torr_2_pascal = torr    # Convert from torr to pascals
pressure = 600 # Units of torr
temperature = 273.15    # Units of Kelvin, STP
stoich = 2 # Stoichiometry of gas

In [ ]:
# Make dictionary containing state specific parameters

states = {}
states["gs"] = {"peak_num": 1,
                "sub_min": -1.0,
                "sub_max": 1.0,
                "cent_min": -0.1,
                "cent_max": 0.6,
                "sim_workspace": Path("E:\\final\\elastic\\workspace\\InterpSolver"),
                "sim_parq": Path("E:\\final\\elastic\\elastic_kine.parquet"),
                "sim_run_min": 0,
                "sim_run_max": 57}

states["3.37"] = {"peak_num": 2,
                "sub_min": 2.8,
                "sub_max": 4.8,
                "cent_min": 2.8,
                "cent_max": 4.8,
                "sim_workspace": Path("E:\\final\\inelastic_3.37\\workspace\\InterpSolver"),
                "sim_parq": Path("E:\\final\\inelastic_3.37\\inelastic_3.37_kine.parquet"),
                "sim_run_min": 0,
                "sim_run_max": 57}

In [ ]:
# Choose state to analyze

state2analyze = "3.37"

In [ ]:
# Create dictionary to store results from analysis

results = {}

# Add keys for result parameters
results["polar"] = np.empty(0, float)
results["kinetic_energy"] = np.empty(0, float)
results["ex_energy"] = np.empty(0, float)
results["cm_polar"] = np.empty(0, float)
results["chisq"] = np.empty(0, float)
results["z_vert"] = np.empty(0, float)

# Create histogram tool
grammer = Histogrammer()

In [ ]:
# Define line shapes

def gaussian(x, amplitude, center, sigma):
    model = lmfit.models.GaussianModel()
    params = model.make_params(amplitude=amplitude, center=center, sigma=sigma)
    return model.eval(params, x=x)

In [ ]:
# Analysis data

target_vector = vector.array({"px": [0.0], "py": [0.0], "pz": [0.0], "E": [target.mass]})
for run in range(run_min, run_max+1):
    df = None
    path = solver_result_path / f"run_{run:04d}_{ejectile.isotopic_symbol}.parquet"
    if not path.exists():
        continue
    df = pl.scan_parquet(path)

    #Apply gates to data
    df = (df.filter((pl.col("vertex_z") >= min_z) 
                   & (pl.col("vertex_z") <= max_z)
                   )
            .collect()
    )

    # Construct the projectile vectors (beam)
    vertices = df.select(['vertex_x', 'vertex_y', 'vertex_z']).to_numpy()
    distances = np.linalg.norm(vertices, axis=1)
    projectile_ke = proj_energy_start - target_material.get_energy_loss(projectile, proj_energy_start, distances)
    projectile_vector = vector.array({
        "px": np.zeros(len(projectile_ke)),
        "py": np.zeros(len(projectile_ke)),
        "pz": np.sqrt(projectile_ke * (projectile_ke + 2.0 * projectile.mass)),
        "E": projectile_ke + projectile.mass
    })

    # Construct the ejectile vectors (detected)
    brho = df.select('brho').to_numpy().flatten()
    momentum = df.select('brho').to_numpy().flatten() * float(ejectile.Z) * QBRHO_2_P
    kinetic_energy = np.sqrt(momentum**2.0 + ejectile.mass**2.0) - ejectile.mass
    polar = df.select('polar').to_numpy().flatten()
    az = df.select('azimuthal').to_numpy().flatten()
    ejectile_vector = vector.array({
        "px": momentum * np.sin(polar) * np.cos(az),
        "py": momentum * np.sin(polar) * np.sin(az),
        "pz": momentum * np.cos(polar),
        "E": np.sqrt(momentum**2.0 + ejectile.mass**2.0)
    })

    # Get reduced chi-squared information
    cs = df.select('redchisq').to_numpy().flatten()

    # Get vertex z position
    zvert=df.select('vertex_z').to_numpy().flatten()

    # Do the kinematics
    residual_vector = target_vector + projectile_vector - ejectile_vector # type: ignore
    ex_energy = residual_vector.mass - residual.mass # Excitation energy is "extra" mass

    # Calculate CM scattering angle 
    cm_vec = ejectile_vector.boostCM_of(projectile_vector + target_vector)
    cm_polar = np.pi - cm_vec.theta

    # Fill histograms and arrays
    results["polar"] = np.append(results["polar"], polar)
    results["kinetic_energy"] = np.append(results["kinetic_energy"], kinetic_energy)
    results["ex_energy"] = np.append(results["ex_energy"], ex_energy)
    results["cm_polar"] = np.append(results["cm_polar"], cm_polar)
    results["chisq"] = np.append(results["chisq"], cs)
    results["z_vert"] = np.append(results["z_vert"], zvert)

In [ ]:
# Plot vertex z-ccordinate histogram

grammer.add_hist1d('z_vert', 1200, (-0.1, 1.1))
grammer.fill_hist1d('z_vert', results["z_vert"])
vertz = grammer.get_hist1d("z_vert")

fig, ax = plt.subplots(1,1)
ax.stairs(vertz.counts, edges=vertz.bins)
ax.set_title(f"Vertex z-coordinate histogram")
ax.set_xlabel("z-coordinate of vertex (m)")
ax.set_ylabel("Counts")
fig.set_figheight(8.0)
fig.set_figwidth(11.0)
fig.tight_layout()

In [ ]:
# Perform analysis of beam events

grammer.add_hist1d('beam_events_ic', 4095, (0.0, 4095.0))

for run in range(run_min, run_max+1):
    df = None
    try:
        path = beam_events_path / f"{form_run_string(run)}.parquet"
        df = pl.read_parquet(path)
    except Exception:
        continue

    # Apply appropriate analysis gates
    df = df.filter((pl.col("ic_multiplicity") == 1)
                 & (pl.col("ic_sca_multiplicity") == 1))
    df = df.filter((abs(pl.col("ic_centroid").list.get(0) - pl.col("ic_sca_centroid").list.get(0)) <= 10))

    ic_amp = df.select(pl.col("ic_amplitude").list.get(0)).to_numpy().flatten()
    grammer.fill_hist1d("beam_events_ic", ic_amp)

In [ ]:
# Plot IC histogram

beam_events_ic = grammer.get_hist1d('beam_events_ic')

beam_nuc1 = lmfit.models.GaussianModel(prefix="beam_nuc1_")
beam_nuc1_bins, beam_nuc1_counts = beam_events_ic.get_subrange((ic_min_val, ic_max_val))
beam_nuc1_bins += 0.5 * beam_events_ic.bin_width
beam_nuc1_weights = np.sqrt(beam_nuc1_counts)
beam_nuc1_weights = np.divide(1, beam_nuc1_weights, out=np.zeros_like(beam_nuc1_weights), where=beam_nuc1_weights!=0.0)
pars = beam_nuc1.guess(x=beam_nuc1_bins, data=beam_nuc1_counts, weights=beam_nuc1_weights)
pars["beam_nuc1_amplitude"].min = 0.0

beam_nuc2 = lmfit.models.GaussianModel(prefix="beam_nuc2_")
beam_nuc2_bins, beam_nuc2_counts = beam_events_ic.get_subrange((ic_max_val, 1500))
beam_nuc2_weights = np.sqrt(beam_nuc2_counts)
beam_nuc2_weights = np.divide(1, beam_nuc2_weights, out=np.zeros_like(beam_nuc2_weights), where=beam_nuc2_weights!=0.0)
pars.update(beam_nuc2.guess(x=beam_nuc2_bins, data=beam_nuc2_counts, weights=beam_nuc2_weights))
pars["beam_nuc2_amplitude"].min = 0.0

total_fit = beam_nuc1 + beam_nuc2
total_bins = beam_events_ic.bins[:-1] + beam_events_ic.bin_width/2
total_counts = beam_events_ic.counts
total_weights = np.sqrt(total_counts)
total_weights = np.divide(1, total_weights, out=np.zeros_like(total_weights), where=total_weights!=0.0)
total_result = total_fit.fit(params=pars, x=total_bins, data=total_counts, weights=total_weights)
comps = total_result.eval_components()

fig, ax = plt.subplots(1,1)
ax.stairs(beam_events_ic.counts, edges=beam_events_ic.bins, label="Spectrum")
ax.plot(total_bins, total_result.best_fit, label="Total Fit")
ax.plot(total_bins, comps["beam_nuc1_"], label="beam_nuc1")
ax.plot(total_bins, comps["beam_nuc2_"], label="beam_nuc2")
ax.set_title('Amplitude of triggering nucleus in beam events')
ax.set_xlabel('Amplitude (ADC units)')
ax.set_ylabel("Counts")
ax.legend()
fig.set_figheight(8.0)
fig.set_figwidth(11.0)
fig.tight_layout()


In [ ]:
# Find counts of beam nucleus

# Find number of contamination counts from neighboring nucleus' peak
A = total_result.params["beam_nuc2_amplitude"].value
mu = total_result.params["beam_nuc2_center"].value
sigma = total_result.params["beam_nuc2_sigma"].value

contamination = quad(gaussian, ic_min_val, ic_max_val, args=(A, mu, sigma))[0] / beam_events_ic.bin_width

# Subtract out contamination from counts
beam_counts, _, _ = beam_events_ic.stats_for_range((ic_min_val, ic_max_val))
beam_counts -= contamination

# Multiply by downscaler
beam_counts *= downscale_factor

In [ ]:
# Calculate kinematics for residual particle in given states
proj_energy_stop = proj_energy_start - target_material.get_energy_loss(projectile, proj_energy_start, np.array([1.0]))[0] # Energy at far end of detector
print(f"Beam energy range: {proj_energy_start}-{proj_energy_stop} MeV")
residual_excitations = np.array([0.0])

# Do the calculation in the center of mass
cm_angle_range = np.linspace(0., np.pi, 1000)
# Calculate envelope from range of beam energies
eject_kinematics = np.zeros((len(residual_excitations), len(cm_angle_range) * 2, 2))

# Upper limit (maximum beam energy)
beam_vec_max = vector.array({
    "px": [0.0],
    "py": [0.0],
    "pz": [np.sqrt(proj_energy_start * (proj_energy_start + 2.0 * projectile.mass))],
    "E": [proj_energy_start + projectile.mass]
})
parent_max = target_vector + beam_vec_max
parent_cm_max = parent_max.boostCM_of(parent_max)
for idx, ex in enumerate(residual_excitations):
    eject_e_cm_max = (ejectile.mass**2.0 - (residual.mass + ex)**2.0 + parent_cm_max.E**2.0) / (2.0 * parent_cm_max.E)
    eject_p_cm_max = np.sqrt(eject_e_cm_max**2.0 - ejectile.mass**2.0)
    eject_vec_cms_max = vector.array({
        "px": eject_p_cm_max * np.sin(cm_angle_range),
        "py": np.zeros(len(cm_angle_range)),
        "pz": eject_p_cm_max * np.cos(cm_angle_range),
        "E": np.full(len(cm_angle_range), eject_e_cm_max)
    })
    eject_vec_lab_max = eject_vec_cms_max.boost(parent_max)
    eject_kinematics[idx, :len(cm_angle_range), 0] = eject_vec_lab_max.theta
    eject_kinematics[idx, :len(cm_angle_range), 1] = eject_vec_lab_max.E - ejectile.mass

# Lower limit (minimum beam energy)
beam_vec_min = vector.array({
    "px": [0.0],
    "py": [0.0],
    "pz": [np.sqrt(proj_energy_stop * (proj_energy_stop + 2.0 * projectile.mass))],
    "E": [proj_energy_stop + projectile.mass]
})
parent_min = target_vector + beam_vec_min
parent_cm_min = parent_min.boostCM_of(parent_min)
for idx, ex in enumerate(residual_excitations):
    eject_e_cm_min = (ejectile.mass**2.0 - (residual.mass + ex)**2.0 + parent_cm_min.E**2.0) / (2.0 * parent_cm_min.E)
    eject_p_cm_min = np.sqrt(eject_e_cm_min**2.0 - ejectile.mass**2.0)
    eject_vec_cms_min = vector.array({
        "px": eject_p_cm_min * np.sin(cm_angle_range),
        "py": np.zeros(len(cm_angle_range)),
        "pz": eject_p_cm_min * np.cos(cm_angle_range),
        "E": np.full(len(cm_angle_range), eject_e_cm_min)
    })
    eject_vec_lab_min = eject_vec_cms_min.boost(parent_min)
    # Note that lower limit is "flipped". We plot the band as a polygon, so need to continue from large lab angles to small lab angles
    eject_kinematics[idx, len(cm_angle_range):, 0] = np.flip(eject_vec_lab_min.theta)
    eject_kinematics[idx, len(cm_angle_range):, 1] = np.flip(eject_vec_lab_min.E - ejectile.mass)

In [ ]:
# Plot kinematics from data and compare with theoretical kinematics
grammer.add_hist2d('ke_theta', (500, 1600), ((0.0, 180.0), (0.0, 80.0)))
grammer.fill_hist2d('ke_theta', np.rad2deg(results["polar"]), results["kinetic_energy"])
ke_theta = grammer.get_hist2d("ke_theta")

fig, ax = plt.subplots(1,1)
mesh = ax.pcolormesh(ke_theta.x_bins, ke_theta.y_bins, ke_theta.counts, norm="log")
ax.set_title("Kinetic Energy vs. Polar Angle")
ax.set_xlabel(r"$\theta$ (deg)")
ax.set_ylabel("Kinetic Energy (MeV)")
fig.colorbar(mesh, ax=ax)
for idx, ex in enumerate(residual_excitations):
    ax.plot(np.rad2deg(eject_kinematics[idx, :len(cm_angle_range), 0]), eject_kinematics[idx, :len(cm_angle_range), 1], label=f"Ex={ex:02} AT-TPC window")
    # Fill idea thanks to D. Ramirez
    ax.fill(np.rad2deg(eject_kinematics[idx, :, 0]), eject_kinematics[idx, :, 1], label=f"Ex={ex:02} Band", alpha=0.2)
ax.legend()
fig.set_figheight(8.0)
fig.set_figwidth(11.0)
fig.tight_layout()

In [ ]:
# Plot reduced chi-squared of fits to data

grammer.add_hist1d('chisq', 2000, (0.0, 1e-4))
grammer.fill_hist1d("chisq", results["chisq"])
hist = grammer.get_hist1d("chisq")

fig, ax = plt.subplots(1,1)
ax.stairs(hist.counts, edges=hist.bins)
ax.set_title(f"{residual.get_latex_rep()} Error")
ax.set_xlabel("Error")
ax.set_ylabel("Counts")
fig.set_figheight(8.0)
fig.set_figwidth(11.0)
fig.tight_layout()

In [ ]:
# Plot excitation energy spectrum

# Gate excitation spectrum on CM angle
mask_cm = (np.deg2rad(min_cm) <= results["cm_polar"]) & (results["cm_polar"] <= np.deg2rad(max_cm))    # Make note of inequalites for binning!
ex_energy_gated = results["ex_energy"][mask_cm]

grammer.add_hist1d("ex_energy",hist_en_bins, (hist_en_min, hist_en_max))
grammer.fill_hist1d("ex_energy", ex_energy_gated)

ex_hist = grammer.get_hist1d("ex_energy")
fig, ax = plt.subplots(1,1)
ax.stairs(ex_hist.counts, edges=ex_hist.bins)
ax.set_title(f"{residual.get_latex_rep()} Spectrum, {min_cm}\N{DEGREE SIGN}-{max_cm}\N{DEGREE SIGN} c.m.")
ax.set_xlabel("Excitation Energy (MeV)")
ax.set_ylabel("Counts")
fig.set_figheight(8.0)
fig.set_figwidth(11.0)
fig.tight_layout()

In [ ]:
# np.savetxt("C:\\Users\\zachs\\Desktop\\transfer_ex_spect.csv", ex_energy_gated, newline="\n", fmt="%.4f", delimiter=",")

In [ ]:
# Fit excitation spectrum peaks for each angular bin

# Create dictionary to store results of excitation background removal
sub_fits = {}

# Make array of edges of angular cuts
ang_bins = np.linspace(min_cm, max_cm, ang_dist_bins+1)

for idx in range(len(ang_bins) - 1):
    low_edge = ang_bins[idx]
    high_edge = ang_bins[idx+1]

    # Gate excitation spectrum on CM angle
    if idx != (len(ang_bins) - 1):
        mask = (np.deg2rad(low_edge) <= results["cm_polar"]) & (results["cm_polar"] < np.deg2rad(high_edge))    # Make note of inequalites for binning!
    else:
        mask = (np.deg2rad(low_edge) <= results["cm_polar"]) & (results["cm_polar"] <= np.deg2rad(high_edge))    # Make note of inequalites for binning!

    ex_energy_gated = results["ex_energy"][mask]

    # Make and fill histogram
    title = f"ex_energy_{low_edge}-{high_edge}"
    grammer.add_hist1d(title, hist_en_bins, (hist_en_min, hist_en_max))
    grammer.fill_hist1d(title, ex_energy_gated)
    sub_ex_hist = grammer.get_hist1d(title)

    # Fit histogram
    peak1 = lmfit.models.GaussianModel(prefix="peak1_")
    peak1_bins, peak1_counts = sub_ex_hist.get_subrange((states["gs"]["sub_min"], states["gs"]["sub_max"]))
    peak1_bins += 0.5 * sub_ex_hist.bin_width
    peak1_weights = np.sqrt(peak1_counts)
    peak1_weights = np.divide(1, peak1_weights, out=np.zeros_like(peak1_weights), where=peak1_weights!=0.0)
    pars = peak1.guess(x=peak1_bins, data=peak1_counts, weights=peak1_weights)
    pars["peak1_amplitude"].min = 0.0
    pars["peak1_center"].min = states["gs"]["cent_min"]
    pars["peak1_center"].max = states["gs"]["cent_max"]

    peak2 = lmfit.models.GaussianModel(prefix="peak2_")
    peak2_bins, peak2_counts = sub_ex_hist.get_subrange((states["3.37"]["sub_min"], states["3.37"]["sub_max"]))
    peak2_bins += 0.5 * sub_ex_hist.bin_width
    peak2_weights = np.sqrt(peak2_counts)
    peak2_weights = np.divide(1, peak2_weights, out=np.zeros_like(peak2_weights), where=peak2_weights!=0.0)
    pars.update(peak2.guess(x=peak2_bins, data=peak2_counts, weights=peak2_weights))
    pars["peak2_amplitude"].min = 0.0
    pars["peak2_center"].min = states["3.37"]["cent_min"]
    pars["peak2_center"].max = states["3.37"]["cent_max"]
    pars["peak1_sigma"].set(expr="peak2_sigma", vary=False)

    total_fit = peak1 + peak2
    total_bins = sub_ex_hist.bins[:-1] + sub_ex_hist.bin_width/2
    total_counts = sub_ex_hist.counts
    total_weights = np.sqrt(total_counts)
    total_weights = np.divide(1, total_weights, out=np.zeros_like(total_weights), where=total_weights!=0.0)
    total_result = total_fit.fit(params=pars, x=total_bins, data=total_counts, weights=total_weights)

    # Store total fit result
    sub_fits[title] = total_result

In [ ]:
# Index for checking fit results
idx = -1

In [ ]:
# Check results of the fits

idx += 1
low_edge = ang_bins[idx]
high_edge = ang_bins[idx+1]
title = f"ex_energy_{low_edge}-{high_edge}"

# Get results of fit
sub_fit_result = sub_fits[title]
comps = sub_fit_result.eval_components()

# Plot result
fig, ax = plt.subplots(1,1)
sub_ex_hist = grammer.get_hist1d(title)
ax.stairs(sub_ex_hist.counts, edges=sub_ex_hist.bins, label="Spectrum")
ax.plot(total_bins, sub_fit_result.best_fit, label="Total Fit")
ax.plot(total_bins, comps["peak1_"], label="peak1")
ax.plot(total_bins, comps["peak2_"], label="peak2")
ax.set_title(f"{residual.get_latex_rep()} Spectrum, {low_edge}\N{DEGREE SIGN}-{high_edge}\N{DEGREE SIGN} c.m.")
ax.set_xlabel("Excitation Energy (MeV)")
ax.set_ylabel("Counts")
ax.legend()
fig.set_figheight(8.0)
fig.set_figwidth(11.0)
fig.tight_layout()

In [ ]:
# Calculate efficiency factors

# Parameters
eff_sim_events_path = states[state2analyze]["sim_workspace"]
kine_path = states[state2analyze]["sim_parq"]
run_min_eff = states[state2analyze]["sim_run_min"]
run_max_eff = states[state2analyze]["sim_run_max"]

# Store results
results_eff = {}
results_eff["cm_polar"] = np.empty(0, float)
results_eff["ex_energy"] = np.empty(0, float)
results_eff["event"] = np.empty(0, int)
results_eff["chisq"] = np.empty(0, float)

for run in range(run_min_eff, run_max_eff+1):
    df = None
    path = eff_sim_events_path / f"run_{run:04d}_{ejectile.isotopic_symbol}.parquet"
    if not path.exists():
        print(f"Run {run} does have a solver file!")
        continue
    df = pl.scan_parquet(path)

    #Apply gates to data
    df = (df.filter((pl.col("vertex_z") >= min_z) 
                   & (pl.col("vertex_z") <= max_z)
                   )
            .collect()
    )

    # Construct the projectile vectors (beam)
    vertices = df.select(['vertex_x', 'vertex_y', 'vertex_z']).to_numpy()
    distances = np.linalg.norm(vertices, axis=1)
    projectile_ke = proj_energy_start - target_material.get_energy_loss(projectile, proj_energy_start, distances)
    projectile_vector = vector.array({
        "px": np.zeros(len(projectile_ke)),
        "py": np.zeros(len(projectile_ke)),
        "pz": np.sqrt(projectile_ke * (projectile_ke + 2.0 * projectile.mass)),
        "E": projectile_ke + projectile.mass
    })

    # Construct the ejectile vectors (detected)
    brho = df.select('brho').to_numpy().flatten()
    momentum = df.select('brho').to_numpy().flatten() * float(ejectile.Z) * QBRHO_2_P
    kinetic_energy = np.sqrt(momentum**2.0 + ejectile.mass**2.0) - ejectile.mass
    polar = df.select('polar').to_numpy().flatten()
    az = df.select('azimuthal').to_numpy().flatten()
    ejectile_vector = vector.array({
        "px": momentum * np.sin(polar) * np.cos(az),
        "py": momentum * np.sin(polar) * np.sin(az),
        "pz": momentum * np.cos(polar),
        "E": np.sqrt(momentum**2.0 + ejectile.mass**2.0)
    })

    # Calculate CM scattering angle 
    cm_vec = ejectile_vector.boostCM_of(projectile_vector + target_vector)
    cm_polar = np.pi - cm_vec.theta

    # Do the kinematics
    residual_vector = target_vector + projectile_vector - ejectile_vector # type: ignore
    ex_energy = residual_vector.mass - residual.mass # Excitation energy is "extra" mass

    # Get reduced chi-squared information
    cs = df.select('redchisq').to_numpy().flatten()

    # Event number
    ev_num = df.select("event").to_numpy()

    # Fill array
    results_eff["cm_polar"] = np.append(results_eff["cm_polar"], cm_polar)
    results_eff["ex_energy"] = np.append(results_eff["ex_energy"], ex_energy)
    results_eff["event"] = np.append(results_eff["event"], ev_num)
    results_eff["chisq"] = np.append(results_eff["chisq"], cs)


# Fit simulated excitation spectrum peak for each angular bin
sub_fits_sim = {}
for idx in range(len(ang_bins) - 1):
    low_edge = ang_bins[idx]
    high_edge = ang_bins[idx+1]

    # Gate excitation spectrum on CM angle
    if idx != (len(ang_bins) - 1):
        mask = (np.deg2rad(low_edge) <= results_eff["cm_polar"]) & (results_eff["cm_polar"] < np.deg2rad(high_edge))    # Make note of inequalites for binning!
    else:
        mask = (np.deg2rad(low_edge) <= results_eff["cm_polar"]) & (results_eff["cm_polar"] <= np.deg2rad(high_edge))    # Make note of inequalites for binning!

    ex_energy_gated = results_eff["ex_energy"][mask]

    # Make and fill histogram
    title = f"sim_ex_energy_{low_edge}-{high_edge}"
    grammer.add_hist1d(title, hist_en_bins, (hist_en_min, hist_en_max))
    grammer.fill_hist1d(title, ex_energy_gated)
    sub_ex_hist = grammer.get_hist1d(title)

    # Fit histogram
    s_peak = lmfit.models.GaussianModel(prefix="s_peak_")
    s_peak_bins, s_peak_counts = sub_ex_hist.get_subrange((states[state2analyze]["sub_min"], states[state2analyze]["sub_max"]))
    s_peak_bins += 0.5 * sub_ex_hist.bin_width
    s_peak_weights = np.sqrt(s_peak_counts)
    s_peak_weights = np.divide(1, s_peak_weights, out=np.zeros_like(s_peak_weights), where=s_peak_weights!=0.0)
    s_pars = s_peak.guess(x=s_peak_bins, data=s_peak_counts, weights=s_peak_weights)
    s_pars["s_peak_amplitude"].min = 0.0
    s_pars["s_peak_center"].min = states[state2analyze]["cent_min"]
    s_pars["s_peak_center"].max = states[state2analyze]["cent_max"]

    s_bins = sub_ex_hist.bins[:-1] + sub_ex_hist.bin_width/2
    s_counts = sub_ex_hist.counts
    s_weights = np.sqrt(s_counts)
    s_weights = np.divide(1, s_weights, out=np.zeros_like(s_weights), where=s_weights!=0.0)
    s_result = s_peak.fit(params=s_pars, x=s_bins, data=s_counts, weights=s_weights)

    # Store total fit result
    sub_fits_sim[title] = s_result

# Find counts in each angular bin
sim_survived = np.zeros(ang_dist_bins, dtype=int)

# Use for Gaussian state
for idx, fit in enumerate(sub_fits_sim.items()):
    amplitude = fit[1].params["s_peak_amplitude"].value
    center= fit[1].params["s_peak_center"].value
    sigma = fit[1].params["s_peak_sigma"].value
    sim_survived[idx] = quad(gaussian, hist_en_min, hist_en_max, args=(amplitude, center, sigma))[0] / ex_hist.bin_width

# Histogram of all simulated CM angles from kinematics file
kine_f = pl.scan_parquet(kine_path)

beam_coords = (
    kine_f.gather_every(4, offset=1)
    .select("px", "py", "pz", "energy")
    .collect()
    .to_numpy()
)
beam_vectors = vector.array(
    {
        "px": beam_coords[:, 0],
        "py": beam_coords[:, 1],
        "pz": beam_coords[:, 2],
        "E": beam_coords[:, 3],
    }
)
product_coords = (
    kine_f.gather_every(4, offset=3)
    .select("px", "py", "pz", "energy")
    .collect()
    .to_numpy()
)
product_vectors = vector.array(
    {
        "px": product_coords[:, 0],
        "py": product_coords[:, 1],
        "pz": product_coords[:, 2],
        "E": product_coords[:, 3],
    }
)

cm_ang_sim_all = product_vectors.boostCM_of(beam_vectors + target_vector).theta
grammer.add_hist1d("cm_polar_sim_all", ang_dist_bins, (min_cm, max_cm))
grammer.fill_hist1d("cm_polar_sim_all", np.rad2deg(cm_ang_sim_all))
sim_all = grammer.get_hist1d("cm_polar_sim_all")

# Calculate efficiency factors
eff_factors = sim_all.counts/sim_survived
eff_factors_err = eff_factors * np.sqrt(1 / sim_all.counts + 1 / sim_survived)

In [ ]:
# Index for checking sim fit results
idx_sim = -1

In [ ]:
# Check results of sim fits

idx_sim += 1
low_edge = ang_bins[idx_sim]
high_edge = ang_bins[idx_sim+1]
title = f"sim_ex_energy_{low_edge}-{high_edge}"

# Get results of fit
sub_sim = sub_fits_sim[title]
comps_sim = sub_sim.eval_components()

# Plot result
fig, ax = plt.subplots(1,1)
sub_ex_hist = grammer.get_hist1d(title)
ax.stairs(sub_ex_hist.counts, edges=sub_ex_hist.bins, label="Spectrum")
ax.plot(s_bins, comps_sim["s_peak_"], label="s_peak_")
ax.set_title(f"{residual.get_latex_rep()} Simulated Spectrum, {low_edge}\N{DEGREE SIGN}-{high_edge}\N{DEGREE SIGN} c.m.")
ax.set_xlabel("Excitation Energy (MeV)")
ax.set_ylabel("Counts")
ax.legend()
fig.set_figheight(8.0)
fig.set_figwidth(11.0)
fig.tight_layout()

In [ ]:
# Find angle error from simulations

# Make LazyFrame of all simulated events' cm angle and their event number
cm_sim_f = pl.LazyFrame({"event": kine_f.gather_every(4).select("event").collect().to_numpy().flatten(), "cm_angle": cm_ang_sim_all})

# Store angle error
cm_err = np.zeros(ang_dist_bins, dtype=float)

for idx in range(len(ang_bins) - 1):
    low_edge = ang_bins[idx]
    high_edge = ang_bins[idx+1]

    # Gate excitation spectrum on CM angle
    if idx != (len(ang_bins) - 1):
        mask_x = (np.deg2rad(low_edge) <= results_eff["cm_polar"]) & (results_eff["cm_polar"] < np.deg2rad(high_edge))    # Make note of inequalites for binning!
    else:
        mask_x = (np.deg2rad(low_edge) <= results_eff["cm_polar"]) & (results_eff["cm_polar"] <= np.deg2rad(high_edge))    # Make note of inequalites for binning!

    # Gate excitation spectrum to only include events with excitation energy in our histogram
    mask_y = (hist_en_min <= results_eff["ex_energy"]) & (results_eff["ex_energy"] <= hist_en_max) 

    # Find events with multiple analyzed tracks
    mult_sort, mult_count = np.unique(results_eff["event"][mask_x & mask_y], return_counts=True)

    # Add events back in
    events_in_bin = np.empty(0, float)
    for mult in range(1, np.max(mult_count)+1):
        sub_frame = cm_sim_f.filter(pl.col("event").is_in(mult_sort[mult_count==mult]))
        sub_frame = pl.concat([sub_frame]*mult)
        events_in_bin = np.append(events_in_bin, sub_frame.select("cm_angle").collect().to_numpy().flatten())

    cm_err[idx] = np.rad2deg(iqr(events_in_bin))/2

In [ ]:
# Find CM scattering angle histogram

cm_counts = np.zeros(ang_dist_bins, dtype=int)

state_id = states[state2analyze]["peak_num"]
# Use for Gaussian states
for idx, fit in enumerate(sub_fits.items()):
    amplitude = fit[1].params[f"peak{state_id}_amplitude"].value
    center= fit[1].params[f"peak{state_id}_center"].value
    sigma = fit[1].params[f"peak{state_id}_sigma"].value
    cm_counts[idx] = quad(gaussian, hist_en_min, hist_en_max, args=(amplitude, center, sigma))[0] / ex_hist.bin_width

grammer.add_hist1d("cm_polar", ang_dist_bins, (min_cm, max_cm))
cm_hist = grammer.get_hist1d("cm_polar")
cm_hist.counts = cm_counts

In [ ]:
# Calculate cross section

length = 1    # Units of meters
nuclei_areal_density = pressure * torr_2_pascal * length / boltzmann / temperature * stoich * 1.0e-31 # Units of target nuclei / millibarns

# # Without efficiency correction
# xs = cm_hist.counts  / np.diff(np.cos(np.deg2rad(cm_hist.bins))) / (-2 * np.pi * beam_counts * nuclei_areal_density)   # Units of millibarns / steradians
# xs_err = np.sqrt(cm_hist.counts) / np.diff(np.cos(np.deg2rad(cm_hist.bins))) / (-2 * np.pi * beam_counts * nuclei_areal_density)

# With effiency correction
xs = cm_hist.counts * eff_factors  / np.diff(np.cos(np.deg2rad(cm_hist.bins))) / (-2 * np.pi * beam_counts * nuclei_areal_density)   # Units of millibarns / steradians
xs_err = np.sqrt((eff_factors * np.sqrt(cm_hist.counts))**2 + (cm_hist.counts * eff_factors_err)**2) / np.diff(np.cos(np.deg2rad(cm_hist.bins))) / (-2 * np.pi * beam_counts * nuclei_areal_density)

In [ ]:
# Plot angular distribution

fig, ax = plt.subplots(1,1)
ax.errorbar(cm_hist.bins[:-1]+cm_hist.bin_width/2, xs, yerr=xs_err, xerr=cm_err, fmt='o', label="This experiment")

#elastic
# elastic_an_cai = pl.read_csv("C:\\Users\\zachs\\Desktop\\elastic_an_cai_93.5.csv")
# elastic_an_cai_d = elastic_an_cai.to_dict(as_series=True)
# ax.plot(elastic_an_cai_d['cm_angle'],elastic_an_cai_d['cm_xs'], label="An Cai")

ax.set_title("Elastic")
ax.set_xlabel(r"$\theta_{CM}$ (deg)")
ax.set_ylabel(r"$d\sigma / d\Omega$ (mb / sr)")
ax.legend()
plt.xlim(min_cm, max_cm)
plt.semilogy()
# plt.ylim(1e-1,1e4)
# plt.ylim(0,20)
# fig.set_figwidth(8.0)
# fig.set_figwidth(11.0)
fig.tight_layout()

In [ ]:
# np.savetxt("C:\\Users\\zachs\\Desktop\\elastic_xs.csv", np.array([cm_hist.bins[:-1]+cm_hist.bin_width/2, cm_err, xs, xs_err]).T, newline="\n", fmt="%.4f", delimiter=",", header="ang(deg),ang_err,xs(mb),xs_err", comments="")